In [221]:
import pandas as pd
df_train = pd.read_csv('train.csv', encoding='utf-8')
df_test = pd.read_csv('test.csv', encoding='utf-8')

In [222]:
import re
def preprocessor(text):
    text = re.sub('[\W]+', ' ', text.lower())
    return text

In [223]:
df_train['text'] = df_train['text'].apply(preprocessor)

In [224]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df_train['author'] = class_le.fit_transform(df_train['author'].values)

In [225]:
from sklearn.model_selection import train_test_split
X, y = df_train.iloc[:, df_train.columns.get_loc('text')].values, \
       df_train.iloc[:, df_train.columns.get_loc('author')].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   stratify=y)

In [226]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords as sw
stopwords = sw.words('english')

[nltk_data] Downloading package stopwords to /home/gg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [227]:
def tokenizer(text):
    return text.split()

In [228]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
def tokenizer_lemma(text):
    return [lemma.lemmatize(word) for word in text.split()]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)

param_grid = {'vect__ngram_range': [(3,3)],
              'vect__stop_words': [stopwords],
              'vect__tokenizer': [tokenizer,
                                  tokenizer_porter,
                                  tokenizer_lemma],
              'vect__norm': ['l1', 'l2', None],
              'vect__use_idf':[False, True],
              'clf__penalty': ['l1', 'l2'],
              'clf__C': [0.0000001]}

pipe = Pipeline([
                 ('vect', tfidf),
                 ('clf', LogisticRegression(random_state=42))
                ])

model = RandomizedSearchCV(pipe, param_grid, scoring='neg_log_loss', cv=10, n_iter=25, verbose=1, n_jobs=-1)
model.fit(X_train, y_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.6s


In [ ]:
print(model.best_estimator_)

In [ ]:
print(model.best_params_)

In [ ]:
print(model.best_score_)

In [ ]:
y_pred_train = model.best_estimator_.predict(X_train)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix_train = confusion_matrix(y_true=y_train, y_pred=y_pred_train)
print(conf_matrix_train)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred_train)

In [ ]:
y_pred_test = model.best_estimator_.predict(X_test)

In [ ]:
conf_matrix_test = confusion_matrix(y_true=y_test, y_pred=y_pred_test)
print(conf_matrix_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_test)

In [ ]:
import pickle
import os
pickle.dump(model.best_estimator_,
           open(os.path.join('./pkl_objects', 'tfidf_3gram_lr_pipe.pkl'), 'wb'),
           protocol=2)